In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Setup plotting
import matplotlib.pyplot as plt
from learntools.deep_learning_intro.dltools import animate_sgd
plt.style.use('seaborn-whitegrid')
# Set Matplotlib defaults
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)
plt.rc('animation', html='html5')

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.model_selection import train_test_split

train = pd.read_csv('../input/tabular-playground-series-feb-2022/train.csv')

X = train.copy()
# Remove target
y = X.pop('target')
X = X.drop(['row_id'], axis=1)

In [ ]:
targets = y.unique()
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(targets)
y = le.transform(y)

In [ ]:
X.head()

In [ ]:
y[:5]

In [ ]:
preprocessor = make_column_transformer(
    (StandardScaler(),
     make_column_selector(dtype_include=np.number)),
    (OneHotEncoder(sparse=False),
     make_column_selector(dtype_include=object)),
)

X = preprocessor.fit_transform(X)

input_shape = [X.shape[1]]
print("Input shape: {}".format(input_shape))

In [ ]:
train.head()

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=input_shape),
    layers.Dense(128, activation='relu'),    
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation="softmax"),
])

# 1) Add Loss and Optimizer

In [ ]:
model.compile(optimizer="rmsprop",
             loss="sparse_categorical_crossentropy",
             metrics=["accuracy"])

In [ ]:
history = model.fit(
    X, y,
    batch_size=128,
    epochs=200,
)

In [ ]:
import pandas as pd

history_df = pd.DataFrame(history.history)
# Start the plot at epoch 5. You can change this to get a different view.
history_df.loc[5:, ['loss']].plot();

# 3) Evaluate Training

In [ ]:
# Experiment with different values for the learning rate, batch size, and number of examples
learning_rate = 0.99
batch_size = 4096
num_examples = 8192

animate_sgd(
    learning_rate=learning_rate,
    batch_size=batch_size,
    num_examples=num_examples,
    # You can also change these, if you like
    steps=50, # total training steps (batches seen)
    true_w=3.0, # the slope of the data
    true_b=2.0, # the bias of the data
)

In [ ]:
test = pd.read_csv('../input/tabular-playground-series-feb-2022/test.csv')

x_test = test.copy()
x_test = x_test.drop(['row_id'], axis=1)
x_test = preprocessor.fit_transform(x_test)

In [ ]:
predictions = model.predict(x_test)

In [ ]:
predictions[0].shape

In [ ]:
np.sum(predictions[0])

In [ ]:
np.argmax(predictions[0])

In [ ]:
max_predictions = [np.argmax(predictions[i]) for i in range(len(predictions))]

In [ ]:
max_predictions[:5]

In [ ]:
bacteria = le.inverse_transform(max_predictions)

In [ ]:
bacteria[:5]

In [ ]:
sample_submission = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2022/sample_submission.csv')
sample_submission.head()

In [ ]:
sample_submission['target'] = bacteria
sample_submission.head()

In [ ]:
sample_submission.to_csv('submission.csv', index=False)